#Libraries


In [402]:
#Libraries
from math import radians, sin, cos, sinh, cosh, sqrt, exp
import math
import pandas as pd
import numpy as np
from io import StringIO
import re
from google.colab import files
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import sheets
from pandas.core.frame import DataFrame
import ipywidgets as widgets
from IPython.display import display
from google.colab import drive
import plotly.express as px
from io import BytesIO
import chardet

# Mount Google Drive

CSV file shoul be in Google Drive to use this option

In [ ]:
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/FilteredData.csv'

file_path = path

##Read CSV

In [ ]:
# Manually decode if necessary (this is usually optional)
def manually_decode_and_read(file_path):
    with open(file_path, 'rb') as f:
        content = f.read()

    encodings = ['utf-8', 'ISO-8859-1', 'latin1']
    for encoding in encodings:
        try:
            decoded_content = content.decode(encoding)
            ts = pd.read_csv(StringIO(decoded_content))
            print(f"Successfully read the file by manually decoding with {encoding}.")
            return ts
        except:
            print(f"Failed to manually decode content with {encoding}.")

    return None

# Attempt to read the file with various encodings
def try_reading_file(file_path):
    encodings = ['utf-8', 'ISO-8859-1', 'latin1']
    for encoding in encodings:
        try:
            ts = pd.read_csv(file_path, encoding=encoding)
            print(f"Successfully read the file with {encoding} encoding.")
            return ts
        except UnicodeDecodeError:
            print(f"Error: Could not read the file with {encoding} encoding.")

    # If reached here, no encoding worked
    print("Failed to read the file with the provided encodings.")
    return None


# First try reading directly
ts = try_reading_file(file_path)

# If that doesn't work, try manual decode (usually optional)
if ts is None:
    ts = manually_decode_and_read(file_path)

# Your subsequent code
if ts is not None:
    ts['Tension'] = pd.NA
    df = ts.copy()  # deep=True by default
    print(ts)
else:
    print("Could not read the file.")

#Browse for file
Get file from your computer.

In [412]:
# Create a FileUpload widget for CSV files
upload_widget = widgets.FileUpload(
    accept='.csv',  # Accept CSV files only
    multiple=False
)

# Use widgets.Output() to capture display output
output_widget = widgets.Output()

# Display the widget
with output_widget:
    display(upload_widget)

# Display the output_widget, which contains the FileUpload widget
display(output_widget)

# Function to read and process the uploaded CSV file(s)
def process_upload(uploaded_files):
    if uploaded_files:
        for filename in uploaded_files.keys():
            content = uploaded_files[filename]['content']

            # Detect the encoding
            result = chardet.detect(content)
            encoding = result['encoding']
            confidence = result['confidence']
            print(f"Detected encoding: {encoding} (Confidence: {confidence})")

            # Read the CSV file using the detected encoding
            data = pd.read_csv(BytesIO(content), encoding=encoding)

            # Clean up the column headers
            data.columns = [re.sub(r'\s+', ' ', col).strip() for col in data.columns]

            # Rename 'Seg #' to 'wh' using flexible matching
            rename_seg_column(data)

            # Rename 'Bend Length (m)' to 'Arc Length' using flexible matching
            rename_bend_length_column(data)

            # Clean the DataFrame by removing rows where all values are NaN
            data = clean_dataframe(data)

            # Clean any invalid values (such as '--' or strange characters)
            data = clean_invalid_values(data)

            return data

# Add the rest of your functions here, unchanged from your original code...

# After uploading, you can process the files by calling:
if upload_widget.value:
    df = process_upload(upload_widget.value)
    if df is not None:
        display(df)
    else:
        print("No data to display. The file might be empty or improperly processed.")
else:
    print("No file uploaded.")


Output()

No file uploaded.


##Assign browsed file to dataframe

In [413]:
# After uploading, you can process the files by calling:
df = process_upload(upload_widget.value)

# Now you can display the DataFrame or do further processing
display(df)

Detected encoding: ISO-8859-1 (Confidence: 0.73)
Renamed 'Seg #' to 'wh'
Renamed 'Bend Length (m)' to 'Arc Length'


<ipython-input-408-b493fc446a94>:109: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,wh,Straight Section Slope (°),Slope Direction,Straight Section Length (m),Bend Type,Bend Direction,Bend Radius (m),Bend Angle (°),Arc Length
0,1,0.0,NaN,8.3,NaN,NaN,0.00,0.0,0.0
1,2,0.0,NaN,6.6,Vertical Up,Up,10.00,1.3,0.2
2,3,0.0,NaN,0.0,Horizontal,NaN,88.94,11.5,17.9
3,4,0.0,NaN,1.7,NaN,NaN,0.00,0.0,0.0
4,5,0.0,NaN,9.9,Horizontal,NaN,62.37,12.6,13.7
5,6,0.0,NaN,16.2,NaN,NaN,0.00,0.0,0.0
6,7,0.0,NaN,72.9,NaN,NaN,0.00,0.0,0.0
7,8,0.0,NaN,105.4,Vertical Down,Down,5.00,9.8,0.9
8,9,9.0,Down,5.2,Vertical Up,Down,5.00,9.3,0.8
9,10,0.0,NaN,0.0,Vertical Down,Down,5.00,9.4,0.8


## Pull Fomulas

In [414]:
def incline_tension_out_down(T_in, W, L, phi_degrees, w, mu):
    """
    Calculate the tension out of a straight section of cable.

    Parameters:
        T_in (float): Tension into the section (Lbs or Kg)
        W (float): Total cable assembly weight per unit of length (Lbs/Ft or Kg/m)
        L (float): Straight section length (feet or meters)
        phi_degrees (float): Straight section angle from horizontal (degrees)
        w (float): Weight correction factor (dimensionless)
        mu (float): Coefficient of friction (dimensionless)

    Returns:
        float: Tension out of the section (Kg)
    """
    T_in = T_in * 101.9716
    phi = radians(phi_degrees)
    T_out = T_in - W * L * (sin(phi) - w * mu * cos(phi))
    T_out = (T_out * 9.80665 / 1000)
    round(T_out, 3)
    return T_out

def incline_tension_out_up(T_in, W, L, phi_degrees, w, mu):
    """
    Calculate the tension out of a straight section of cable, Returns in kN.

    Parameters:
        T_in (float): Tension into the section (kgs)
        W (float): Total cable assembly weight per unit of length (kg/m)
        L (float): Straight section length (meters)
        phi_degrees (float): Straight section angle from horizontal (degrees)
        w (float): Weight correction factor (dimensionless)
        mu (float): Coefficient of friction (dimensionless)

    Returns:
        float: Tension out of the section (kN)
    """
    T_in = T_in * 101.9716
    phi = radians(phi_degrees)
    T_out = T_in + W * L * (sin(phi) + w * mu * cos(phi))
    T_out = (T_out * 9.80665 / 1000)
    T_out = round(T_out, 3)
    return T_out ## kN

def horizontal_bend_section(T_in, w, W, theta_degrees, R, mu):
  """
   Calculate the tension out of a horizontal bend section of cable.

   Parameters:
       T_in (float): Tension into the section (Kg)
       w (float): Weight correction factor (dimensionless)
       W (float): Total cable assembly weight per unit of length (Kg/m)
       theta_degrees (float): Bend section angle (degrees)
       R(float): Bend section Radious
       mu (float): Coefficient of friction (dimensionless)
  """
  T_in = T_in * 101.9716
  theta = radians(theta_degrees)
  T_out = T_in * cosh(w * mu * theta) + sinh(w *mu * theta) * sqrt((T_in ** 2) + (W * R)**2)
  T_out = (T_out * 9.80665 / 1000)
  T_out = round(T_out, 3)

  return T_out ##kN


def tension_out_vertical_bend_up_pulling_up(T_in, w, mu, theta_degrees, R, W):
    """
    Calculate the tension out of a vertical bend section pulling up.

    Parameters:
        T_in (float): Tension into the section (Kg)
        w (float): Weight correction factor (dimensionless)
        mu (float): Coefficient of friction (dimensionless)
        theta_degrees (float): Bend section angle (degrees)
        R (float): Bend section radius (feet or meters)
        W (float): Total cable assembly weight per unit of length (Kg/m)

    Returns:
        float: Tension out of the section (Lbs or Kg)
    """
    # Convert angle to radians for trigonometric calculations
    theta = radians(theta_degrees)
    T_in = T_in * 101.9716

    # Calculate the tension out of the section
    T_out = T_in * math.exp(w * mu * theta) - \
            (W * R) / (1 + (w * mu)**2) * \
            ((2 * w * mu * sin(theta) - (1 - (w * mu)**2)) * (math.exp(w * mu * theta) - cos(theta)))


    T_out = (T_out * 9.80665 / 1000)
    T_out = round(T_out, 3)

    return T_out

def tension_out_vertical_bend_up_pulling_down(T_in, w, mu, theta_degrees, R, W):
    """
    Calculate the tension out of a vertical bend section pulling up.

    Parameters:
        T_in (float): Tension into the section (Lbs or Kg)
        w (float): Weight correction factor (dimensionless)
        mu (float): Coefficient of friction (dimensionless)
        theta_degrees (float): Bend section angle (degrees)
        R (float): Bend section radius (feet or meters)

    Returns:
        float: Tension out of the section (Lbs or Kg)
    """
    # Convert angle to radians for trigonometric calculations
    theta = radians(theta_degrees)
    T_in = T_in * 101.9716

    # Calculate the tension out of the section
    T_out = T_in * math.exp(w * mu * theta) - \
            (W * R) / (1 + (w * mu)**2) * \
            (2 * w * mu * math.exp(w * mu * theta)* sin(theta)) + (1 - (w * mu)**2) * (1 - (math.exp(w * mu * theta) * cos(theta)))
    T_out = (T_out * 9.80665 / 1000)
    T_out = round(T_out, 3)
    return T_out


def tension_out_vertical_bend_down_pulling_up(T_in, w, mu, theta_degrees, R, W):
    """
    Calculate the tension out of a vertical bend section pulling up.

    Parameters:
        T_in (float): Tension into the section (Lbs or Kg)
        w (float): Weight correction factor (dimensionless)
        mu (float): Coefficient of friction (dimensionless)
        theta_degrees (float): Bend section angle (degrees)
        R (float): Bend section radius (feet or meters)

    Returns:
        float: Tension out of the section (Lbs or Kg)
    """
    # Convert angle to radians for trigonometric calculations
    theta = radians(theta_degrees)
    T_in = T_in * 101.9716

    # Calculate the tension out of the section
    T_out = T_in * math.exp(w * mu * theta) + \
            (W * R) / (1 + (w * mu)**2) * \
            (2 * w * mu * math.exp(w * mu * theta) * sin(theta)) + (1 - (w * mu)**2) * (1 - (math.exp(w * mu * theta) * cos(theta)))
    T_out = (T_out * 9.80665 / 1000)
    T_out = round(T_out, 3)

    return T_out

def tension_out_vertical_bend_down_pulling_down(T_in, w, mu, theta_degrees, R,W):
    """
    Calculate the tension out of a vertical bend section pulling up.

    Parameters:
        T_in (float): Tension into the section (Lbs or Kg)
        w (float): Weight correction factor (dimensionless)
        mu (float): Coefficient of friction (dimensionless)
        theta_degrees (float): Bend section angle (degrees)
        R (float): Bend section radius (feet or meters)

    Returns:
        float: Tension out of the section (Lbs or Kg)
    """
    # Convert angle to radians for trigonometric calculations
    theta = radians(theta_degrees)
    T_in = T_in * 101.9716

    # Calculate the tension out of the section
    T_out = T_in * math.exp(w * mu * theta) + \
            (W * R) / (1 + (w * mu)**2) * \
            (2 * w * mu * sin(theta) - (1 - (w * mu)**2) * (math.exp(w * mu * theta) - cos(theta)))
    T_out = (T_out * 9.80665 / 1000)
    T_out = round(T_out, 3)

    return T_out


##Main Logic

In [415]:
def calculate_tension(row, Back_Tension, w, W, i, ts):
    T_in = 0
    T = 0
    slope = row['Straight Section Slope (°)']
    direction = row['Slope Direction']
    length = row['Straight Section Length (m)']
    is_slope_zero_or_nan = slope == 0 or pd.isna(slope)
    is_direction_nan = pd.isna(direction)
    is_length_zero_or_nan = length == 0 or pd.isna(length)

  ###################### INcline Cases #################################

    if is_slope_zero_or_nan and is_direction_nan and not is_length_zero_or_nan:
        mu = row['Straight Section COF']
        if i == 0:
            T = round((Back_Tension), 3)  ## Output kN
        else:
            T = round((ts.at[i-1, 'Tension']), 3)
        T_in = (w * mu * W * length)*9.81/1000 + T  ## Output kN

    elif not is_slope_zero_or_nan:
        mu = row['Straight Section COF']
        phi_degrees = slope
        if direction == 'Up':
            T_in = incline_tension_out_up(((ts.at[i-1, 'Tension'])), W, length, phi_degrees, w, mu) ## Output kN
        else:
            T_in = incline_tension_out_down(((ts.at[i-1, 'Tension'])), W, length, phi_degrees, w, mu) ## Output kN
    if i == 0 and is_slope_zero_or_nan and is_direction_nan and is_length_zero_or_nan:
        T_in = round((Back_Tension), 3) ## Output kN
    elif i != 0 and is_slope_zero_or_nan and is_direction_nan and is_length_zero_or_nan:
        T_in = round((ts.at[i-1, 'Tension']), 3) ## Output kN

    if (row['Bend Type'] == 0 or row['Bend Type'] is None or pd.isna(row['Bend Type'])) and not is_slope_zero_or_nan:
        return round((T_in), 3)

  ###################### INcline Cases #################################

  ###################### Bends #################################

    if row['Bend Type'] == 'Horizontal':
        mu = row['Bend COF']
        theta_degrees = row['Bend Angle (°)']
        R = row['Bend Radius (m)']
        return horizontal_bend_section(T_in, w, W, theta_degrees, R, mu)

    elif row['Bend Type'] == 'Vertical Up':
        mu = row['Bend COF']
        theta_degrees = row['Bend Angle (°)']
        R = row['Bend Radius (m)']
        if row['Bend Direction'] == 'Up':
            return tension_out_vertical_bend_up_pulling_up(T_in, w, mu, theta_degrees, R, W)
        return tension_out_vertical_bend_up_pulling_down(T_in, w, mu, theta_degrees, R, W)

    elif row['Bend Type'] == 'Vertical Down':
        mu = row['Bend COF']
        theta_degrees = row['Bend Angle (°)']
        R = row['Bend Radius (m)']
        if row['Bend Direction'] == 'Down':
            return tension_out_vertical_bend_down_pulling_down(T_in, w, mu, theta_degrees, R, W)
        return tension_out_vertical_bend_down_pulling_up(T_in, w, mu, theta_degrees, R, W)

  ###################### Bends #################################

    return round((T_in), 3)

## General Functions

In [416]:
def get_max_tension():
    while True:
        try:
            Max_Tension = float(input("Please enter the Maximum Allowable Tension in kN: "))
            if Max_Tension <= 0:
                print("Please enter a positive number.")
                continue
            print(f"Stored Maximum Allowable Tension: {Max_Tension} kN")
            return Max_Tension
        except ValueError:
            print("Invalid input. Please enter a numerical value.")


def check_tension(df, Max_Tension, Back_Tension, w, mu, W):
    df_len = len(df)  # Data set total rows
    j = 0  # Loop counter for rows
    i = 0  # Additional counter to keep track of previous row

    for j in range(df_len):  # This will loop through all rows
        # Pass the additional parameters to the calculate_tension function
        calculated_tension = calculate_tension(df.iloc[j], Back_Tension, w, mu, W, i, df)
        df.at[j, 'Tension'] = calculated_tension

        # Check if the calculated tension exceeds Max_Tension
        if calculated_tension >= Max_Tension:
            return "Cable Pusher Needed"

        i += 1  # Update i for the next iteration

    return "All tensions are within the limit"


def update_joint_bay_variable(ts, Max_Tension_Row):
    i = 0
    Joint_Bay = ''  # Initialize Joint_Bay here

    while i <= Max_Tension_Row - 1:

        is_Joint_Bay_Row_empty = pd.isna(ts.at[i, 'Joint Bay'])

        if i == 0:
            Joint_Bay = ts.at[0, 'Joint Bay']
        elif is_Joint_Bay_Row_empty == False:
            Joint_Bay = ts.at[i, 'Joint Bay']

        i += 1  # Increment i to avoid infinite loop

    return Joint_Bay

def reverse_values(reversed_ts):
    i = 0
    while i < len(reversed_ts):

        if reversed_ts.at[i, 'Slope Direction'] == 'Up':
            reversed_ts.at[i, 'Slope Direction'] = 'Down'

        elif reversed_ts.at[i, 'Slope Direction'] == 'Down':
            reversed_ts.at[i, 'Slope Direction'] = 'Up'

        if reversed_ts.at[i, 'Bend Type'] == 'Vertical Up':
            reversed_ts.at[i, 'Bend Type'] = 'Vertical Up'

        elif reversed_ts.at[i, 'Bend Type'] == 'Vertical Down':
            reversed_ts.at[i, 'Bend Type'] = 'Vertical Down'

        if reversed_ts.at[i, 'Bend Direction'] == 'Up':
            reversed_ts.at[i, 'Bend Direction'] = 'Down'

        elif reversed_ts.at[i, 'Bend Direction'] == 'Down':
            reversed_ts.at[i, 'Bend Direction'] = 'Up'

        i += 1

    return reversed_ts

def reverse_dataframe(df):
    reversed_df = df.iloc[::-1].reset_index(drop=True)
    reversed_df['Tension'] = np.nan
    return reversed_df

def update_tension_and_pressure(df, Back_Tension, w, W, Cable_Configuration):
    j = 0
    i = 0
    for j in range(len(df)):
        df.at[j, 'Tension'] = calculate_tension(df.iloc[j], Back_Tension, w, W, i, df)
        i += 1
    add_side_wall_pressure(df, Cable_Configuration, w)
    return df

def copy_columns_based_on_wh(df_source, df_target):
    # Initialize the columns with NaN (more standard for missing values in DataFrames)
    df_target['Reverse Tension'] = float('nan')
    df_target['Reverse Side Wall Pressure'] = float('nan')

    # Iterate through the source DataFrame rows
    for i, row in df_source.iterrows():
        wh_value = row['wh']  # Get the 'wh' value from the current row
        target_idx_list = df_target[df_target['wh'] == wh_value].index.tolist()  # Find matching indices

        # Assign values to the target DataFrame
        for target_idx in target_idx_list:
            df_target.at[target_idx, 'Reverse Tension'] = row['Tension']
            df_target.at[target_idx, 'Reverse Side Wall Pressure'] = row['Side Wall Pressure']

    return df_target

def reverse_values(reversed_ts):
    i = 0
    while i < len(reversed_ts):
        if reversed_ts.at[i, 'Slope Direction'] == 'Up':
            reversed_ts.at[i, 'Slope Direction'] = 'Down'
        elif reversed_ts.at[i, 'Slope Direction'] == 'Down':
            reversed_ts.at[i, 'Slope Direction'] = 'Up'

        if reversed_ts.at[i, 'Bend Type'] == 'Vertical Up':
            reversed_ts.at[i, 'Bend Type'] = 'Vertical Up'
        elif reversed_ts.at[i, 'Bend Type'] == 'Vertical Down':
            reversed_ts.at[i, 'Bend Type'] = 'Vertical Down'

        if reversed_ts.at[i, 'Bend Direction'] == 'Up':
            reversed_ts.at[i, 'Bend Direction'] = 'Down'
        elif reversed_ts.at[i, 'Bend Direction'] == 'Down':
            reversed_ts.at[i, 'Bend Direction'] = 'Up'

        i += 1

    return reversed_ts

def combined_function(subset_0, Back_Tension, w, W, Cable_Configuration):

    subset_0 = update_tension_and_pressure(subset_0, Back_Tension, w, W, Cable_Configuration)

    # Reverse the dataframe
    r_subset_0 = reverse_dataframe(subset_0)

    # Reverse values for specific columns
    r_subset_0 = reverse_values(r_subset_0)

    # Update tension and pressure
    r_subset_0 = update_tension_and_pressure_reverse_case(r_subset_0, Back_Tension, w, W, Cable_Configuration)

    # Copy columns based on 'wh'
    final_subset_0 = copy_columns_based_on_wh(r_subset_0, subset_0)

    return final_subset_0


def update_direction_column(df):
    df['Direction'] = '|'
    df.loc[~pd.isna(df['Joint Bay']), 'Direction'] = '^'

    # Fill NaN with the first element so that row 0 is properly evaluated
    shifted_tension = df['Tension'].shift().fillna(df['Tension'].iloc[0])

    df.loc[(df['Direction'] == '^') & (shifted_tension <= df['Tension']), 'Direction'] = 'v'

    if df.at[1, 'Tension'] < df.at[0, 'Tension']:
        df.at[0, 'Direction'] = '^'

    j = 0
    for j in range(len(df)):
        if df.at[j, 'Tension'] == 'Cable Pusher Needed':
            df.at[j, 'Direction'] = 'Add Cable Pusher'

    return df

def download_dataframe(df):
    while True:
        # Prompt the user to decide whether to download the file
        user_input = input("Do you want to download the file? Type 'y' for yes or 'n' for no: ")

        # Convert the input to lower case for case-insensitive comparison
        user_input = user_input.lower()

        if user_input == 'y':
            # To save the DataFrame to a CSV file in the Colab environment
            df.to_csv('df.csv', index=False)

            # To download the CSV file to your local machine
            files.download('df.csv')
            break  # Exit the loop

        elif user_input == 'n':
            print("File will not be downloaded.")
            break  # Exit the loop

        else:
            print("Invalid input. Type 'y' for yes or 'n' for no.")

def calculate_length_add_previous(df): # Change parameter name to df
    # Replace NaN with 0 for the calculation
    straight_section_length = df['Straight Section Length (m)'].fillna(0)
    bend_radius = df['Bend Radius (m)'].fillna(0)
    bend_angle_deg = df['Bend Angle (°)'].fillna(0) # Assuming 'Bend Angle (°)' is a column in df

    # Convert the 'Bend Angle (°)' to radians
    bend_angle_rad = np.radians(bend_angle_deg)

    # Calculate the length
    length = straight_section_length + (bend_radius * bend_angle_rad)

    # Perform a cumulative sum to add the value from the row above
    df['Length (m)'] = length.cumsum()

    # Round the 'Length (m)' column to 3 decimal places
    df['Length (m)'] = df['Length (m)']

    return df

def get_diameter_input():
    while True:
        try:
            dim = float(input("Please enter the cable diameter in mm: "))
            return dim
        except ValueError:
            print("Invalid input. Please enter a numerical value for the diameter.")

def create_and_clean_subset(ts):
    # Function to prompt for joint bays and handle invalid input
    def prompt_for_joint_bays(ts):
        joint_bay_list = ts['Joint Bay'].dropna().tolist()
        print("Available Joint Bays:", joint_bay_list)
        start_joint_bay = input("Enter the starting Joint Bay: ")
        end_joint_bay = input("Enter the ending Joint Bay: ")

        if start_joint_bay not in joint_bay_list or end_joint_bay not in joint_bay_list:
            print("Invalid Joint Bay selection.")
            return None

        if joint_bay_list.index(start_joint_bay) > joint_bay_list.index(end_joint_bay):
            print("Invalid range: The starting Joint Bay appears after the ending Joint Bay in the list.")
            return None

        return (start_joint_bay, end_joint_bay)

    # Prompt the user for the joint bays to be used
    selected_joint_bays = prompt_for_joint_bays(ts)
    if selected_joint_bays is None:
        return None

    start_joint_bay, end_joint_bay = selected_joint_bays

    # Filter the DataFrame based on the selected joint bays
    start_index = ts[ts['Joint Bay'] == start_joint_bay].index[0]
    end_index = ts[ts['Joint Bay'] == end_joint_bay].index[-1]

    subset = ts.loc[start_index:end_index].copy()

    return subset

def suggested_back_tension(df, W):
    # Get the last value from the "Length (m)" column
    last_length = df["Length (m)"].iloc[-1]

    # Prompt the user for the percentage of total weight they would like to use
    percentage = float(input(f"Enter the percentage of total weight for back tension (e.g., 3 for 3%): "))

    # Calculate the Back Tension and multiply by the last length value
    Back_Tension = (percentage / 100) * W * 9.81 / 1000 * last_length
    Back_Tension = round(Back_Tension, 2)

    # Print or return the Back Tension value
    print(f"Suggested Back Tension: {Back_Tension} kN based on the total length value of {last_length} m")

    return Back_Tension

def adjust_back_tension_for_cable_pusher(Back_Tension):
    # Prompt the user for the cable pusher value in kN
    Cable_Pusher = float(input("Enter the value for the cable pusher in kN: "))
    Cable_Pusher = round(Cable_Pusher, 2)

    # Adjust the Back Tension by subtracting the Cable Pusher value
    Back_Tension = Back_Tension - Cable_Pusher

    # Round the adjusted Back Tension value to two decimal places
    Back_Tension = round(Back_Tension, 2)

    return Back_Tension, Cable_Pusher

def get_cof_and_source(cof_data_base):
    # Extract unique materials from the DataFrame
    duct_materials = cof_data_base['Duct Materal'].unique()
    cable_outer_jackets = cof_data_base['Cable Outer Jacket'].unique()

    # Create dropdown widgets for Duct Material and Cable Outer Jacket Material
    duct_material_dropdown = widgets.Dropdown(
        options=duct_materials,
        description='Duct Material:'
    )

    cable_outer_jacket_dropdown = widgets.Dropdown(
        options=cable_outer_jackets,
        description='Cable Material:'
    )

    # Display the dropdown widgets
    display(duct_material_dropdown)
    display(cable_outer_jacket_dropdown)

    def on_button_click(b):
        duct_material = duct_material_dropdown.value
        cable_outer_jacket = cable_outer_jacket_dropdown.value

        # Filter the DataFrame based on user input without converting to lowercase
        result = cof_data_base[
            (cof_data_base['Duct Materal'] == duct_material) &
            (cof_data_base['Cable Outer Jacket'] == cable_outer_jacket)
        ]

        # Check if a match was found
        if not result.empty:
            cof_value = result.iloc[0]['COF']
            source_value = result.iloc[0]['Source']
            print(f"COF: {cof_value}, Duct Material: {duct_material}, Cable Material: {cable_outer_jacket}, Source: {source_value}")
        else:
            print("No matching data found for the given Duct Material and Cable Outer Jacket Material.")

    # Create and display a button to submit the selections
    button = widgets.Button(description="Get COF and Source")
    button.on_click(on_button_click)
    display(button)
########################################################################################################### Target COF#########################################
def adjust_cof_to_target_tension(df, combined_function, Back_Tension, w, W, Cable_Configuration):
    # Ask the user for the target tension value and direction
    target_tension = float(input("Please enter the target tension value: "))
    direction = input("Do you want to target 'Tension' (forward pull) or 'Reverse Tension'? ").strip().lower()

    # Set the initial COF values
    df['Straight Section COF'] = 0.01
    df['Bend COF'] = 0.01

    tension_column = 'Tension' if direction == 'tension' else 'Reverse Tension'

    # Iterate to find the appropriate COF values
    while True:
        # Apply the combined function to the DataFrame
        df = combined_function(df, Back_Tension, w, W, Cable_Configuration)

        # Get the tension value to compare against the target
        if direction == 'tension':
            tension_value = df[tension_column].iloc[-1]  # last row for forward pull
        else:
            tension_value = df[tension_column].iloc[0]  # first row for reverse pull

        # Check if the tension value meets or exceeds the target tension
        if tension_value >= target_tension:
            break

        # Increase the COF values by 0.01
        df['Straight Section COF'] += 0.01
        df['Bend COF'] += 0.01

        # Stop if COF values reach the maximum limit of 0.60 (to prevent infinite loop)
        if df['Straight Section COF'].iloc[0] > 0.60 or df['Bend COF'].iloc[0] > 0.60:
            print("COF values have reached the maximum limit (0.60). Exiting the loop.")
            break

    print(f"Target tension reached with Straight Section COF: {df['Straight Section COF'].iloc[0]:.2f}, Bend COF: {df['Bend COF'].iloc[0]:.2f}")

    return df
###################################################################################################################Target COF#########################################
def get_max_side_wall_pressure():
    try:
        # Prompt the user for input in kN/m
        pressure_kn_m = float(input("Enter the Max Side Wall Pressure (in kN/m): "))

        # Display the entered pressure
        print(f"\nThe Max Side Wall Pressure is {pressure_kn_m:.2f} kN/m")

        # Return the pressure for further use if needed
        return pressure_kn_m
    except ValueError:
        print("Please enter a valid numerical value.")
################################################################################################################ NOTES ###################################################################################################################

def add_notes_column(df, Cable_Pusher, Lubricant, Max_Tension, Max_SWP, w, Cable_Configuration, additional_notes=None):
    # Create the initial notes
    notes = [
        f"Cable Pusher: {Cable_Pusher} kN",  # First note
        f"Lubricant: {Lubricant} lts"        # Second note
    ]

    # Determine the maximum value between "Side Wall Pressure" and "Reverse Side Wall Pressure" columns, ignoring inf
    if 'Side Wall Pressure' in df.columns and 'Reverse Side Wall Pressure' in df.columns:
        # Replace inf values with NaN to ignore them
        side_wall_pressure = df['Side Wall Pressure'].replace([np.inf, -np.inf], np.nan)
        reverse_side_wall_pressure = df['Reverse Side Wall Pressure'].replace([np.inf, -np.inf], np.nan)

        # Calculate the maximum value ignoring NaN (which replaced inf)
        max_pressure =round(max(side_wall_pressure.max(), reverse_side_wall_pressure.max()), 3)

        if pd.notna(max_pressure):
            notes.append(f"Max Side Wall Pressure: {max_pressure} kN/m")  # Third note

            # Calculate the Max SWP Utilization Factor
            max_swp_utilization_factor = round(100 * max_pressure / Max_SWP, 2)
            notes.append(f"Max SWP Utilization Factor: {max_swp_utilization_factor} %")
        else:
            notes.append("Max Side Wall Pressure: Not available")
            notes.append("")  # Empty note for SWP Utilization Factor
    else:
        notes.extend(['', ''])  # Add empty notes if columns are missing

    # Add the Max Forward Tension note from the last row of the "Tension" column
    if 'Tension' in df.columns:
        max_forward_tension = df['Tension'].iloc[-1]
        notes.append(f"Max Forward Tension: {max_forward_tension} kN")

        # Calculate the Utilization Factor for Forward Tension
        utilization_factor = round(100 * max_forward_tension / Max_Tension, 2)
        notes.append(f"Forward Utilization Factor: {utilization_factor} %")
    else:
        notes.extend(['', ''])  # Add empty notes if "Tension" column is missing

    # Add the Max Reverse Tension note from the first row of the "Reverse Tension" column
    if 'Reverse Tension' in df.columns:
        max_reverse_tension = df['Reverse Tension'].iloc[0]
        notes.append(f"Max Reverse Tension: {max_reverse_tension} kN")

        # Calculate the Utilization Factor for Reverse Tension
        utilization_factor_reverse = round(100 * max_reverse_tension / Max_Tension, 2)
        notes.append(f"Reverse Utilization Factor: {utilization_factor_reverse} %")
    else:
        notes.extend(['', ''])  # Add empty notes if "Reverse Tension" column is missing

    # Add notes about the WEIGHT CORRELATION FACTOR and Cable Configuration
    notes.append(f"Weight Correlation Factor: {w}")
    notes.append(f"Cable Configuration: {Cable_Configuration}")

    # Add any additional notes provided by the user
    if additional_notes:
        notes.extend(additional_notes)

    # Ensure the notes list matches the length of the DataFrame
    if len(notes) < len(df):
        notes += [''] * (len(df) - len(notes))
    else:
        notes = notes[:len(df)]  # Trim the list if there are more notes than rows

    # Add the "Notes" column to the DataFrame
    df['Notes'] = notes

    return df


############################################################################################################### NOTES ###################################################################################################################

############################################################################################################### WFC ####################################
def calculate_wcf():
    # Create input widgets
    duct_diameter_input = widgets.FloatText(description="Duct Diameter [D] (mm):", style={'description_width': 'initial'})
    cable_diameter_input = widgets.FloatText(description="Cable Diameter [d] (mm):", style={'description_width': 'initial'})

    # Define the dropdown menu for cable configuration
    config_dropdown = widgets.Dropdown(
        options=[
            ('1 Cable', 1),
            ('2 Cables', 2),
            ('3 Cables (Triangular)', 3),
            ('3 Cables (Cradled)', 4),
            ('4 Cables or more', 5)
        ],
        description='Configuration:',
        style={'description_width': 'initial'}
    )

    # Create Calculate button
    calculate_button = widgets.Button(description="Calculate WCF")

    # Create output widget
    output = widgets.Output()

    # Variables to store the outputs
    User_inputs = [None, None, None, None]  # Placeholder for storing outputs

    def calculate_and_display_w(b):
        with output:
            output.clear_output()

            try:
                # Get values from widgets
                Duct_Diameter = duct_diameter_input.value
                Cable_Diameter = cable_diameter_input.value
                config = config_dropdown.value

                # Validate input values
                if Duct_Diameter <= 0 or Cable_Diameter <= 0:
                    print("Error: Diameters must be positive numbers.")
                    return

                if config == 1:
                    w_value = 1
                    config_name = "1 Cable"
                elif config in [2, 3]:
                    ratio = Cable_Diameter / (Duct_Diameter - Cable_Diameter)
                    if ratio >= 1:
                        print("Error: The ratio of d/(D-d) is too large, which results in an invalid calculation.")
                        return
                    w_value = 1 / math.sqrt(1 - ratio**2)
                    config_name = "2 Cables" if config == 2 else "3 Cables (Triangular)"
                elif config == 4:
                    ratio = Cable_Diameter / (Duct_Diameter - Cable_Diameter)
                    if ratio >= 1:
                        print("Error: The ratio of d/(D-d) is too large, which results in an invalid calculation.")
                        return
                    w_value = 1 + (4/3) * ratio**2
                    config_name = "3 Cables (Cradled)"
                elif config == 5:
                    w_value = 1.4
                    config_name = "4 Cables or more"

                # Round w_value to 2 decimal places
                w_value = round(w_value, 2)

                # Display the result and the configuration
                print(f"Duct Diameter: {Duct_Diameter} mm")
                print(f"Cable Diameter: {Cable_Diameter} mm")
                print(f"Configuration selected: {config_name}")
                print(f"The value of w is: {w_value:.2f}")

                # Store the result
                User_inputs[0] = w_value
                User_inputs[1] = config_name
                User_inputs[2] = Duct_Diameter
                User_inputs[3] = Cable_Diameter

            except Exception as e:
                print(f"An error occurred: {str(e)}")

    # Link the button to the calculation function
    calculate_button.on_click(calculate_and_display_w)

    # Display all widgets
    display(duct_diameter_input, cable_diameter_input, config_dropdown, calculate_button, output)

    return User_inputs  # Return the list to store the result

# Call the function

############################################################################################################### WFC ####################################

def add_side_wall_pressure(ts, cable_configuration, w):
    """
    Add a new column to the ts DataFrame to calculate Side Wall Pressure.
    The calculation depends on the cable configuration input by the user.

    Parameters:
        ts (pd.DataFrame): The DataFrame containing 'Tension', 'Bend Radius (m)',
                           and 'Cable Width' columns.
        cable_configuration (str): The cable configuration selected by the user.

    Returns:
        pd.DataFrame: The updated DataFrame with the new 'Side Wall Pressure' column.
    """
    # Check if required columns exist in the DataFrame
    required_columns = ['Tension', 'Bend Radius (m)']
    for col in required_columns:
        if col not in ts.columns:
            raise ValueError(f"The '{col}' column must exist in the DataFrame.")

    # Convert columns to numeric type to ensure calculations and rounding will work
    ts['Tension'] = pd.to_numeric(ts['Tension'], errors='coerce')
    ts['Bend Radius (m)'] = pd.to_numeric(ts['Bend Radius (m)'], errors='coerce')

    # Calculate Side Wall Pressure based on the cable configuration input by the user
    def calculate_sp(row):
        T = row['Tension']
        R = row['Bend Radius (m)']

        if pd.isna(R) or R == 0:
            return np.nan  # Avoid division by zero

        if cable_configuration == '1 Cable':
            return (T / R)
        elif cable_configuration == '2 Cables':
            return ((w / 2) * T / R)
        elif cable_configuration == '3 Cables (Cradled)':
            return (((3 * w - 2) / 3) * T / R)
        elif cable_configuration == '3 Cables (Triangular)':
            return ((w / 2) * T / R)
        elif cable_configuration == '4 Cables or more':
            return ((w / 2) * T / R)
        else:
            return np.nan  # Handle unknown configurations

    ts['Side Wall Pressure'] = ts.apply(calculate_sp, axis=1)

    return ts

def round_to_two_decimals(df):
    """
    This function takes a DataFrame and rounds all numeric columns to two decimal places.

    Parameters:
    df (pandas.DataFrame): The DataFrame whose numeric columns need to be rounded.

    Returns:
    pandas.DataFrame: The DataFrame with all numeric columns rounded to two decimal places.
    """
    return df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

######################################################################################## reverse case

def calculate_tension_reverse_case(row, Back_Tension, w, W, i, ts):
    T_in = 0
    T = 0
    slope = row['Straight Section Slope (°)']
    direction = row['Slope Direction']
    length = row['Straight Section Length (m)']
    is_slope_zero_or_nan = slope == 0 or pd.isna(slope)
    is_direction_nan = pd.isna(direction) | (direction == 0)
    is_length_zero_or_nan = length == 0 or pd.isna(length)

  ###################### Bends #################################
    if i == 0:
            T_in = round((Back_Tension), 3)  ## Output kN
    else:
            T_in = round((ts.at[i-1, 'Tension']), 3)

    if row['Bend Type'] == 'Horizontal':
        mu = row['Bend COF']
        theta_degrees = row['Bend Angle (°)']
        R = row['Bend Radius (m)']
        T_in = horizontal_bend_section(T_in, w, W, theta_degrees, R, mu)

    elif row['Bend Type'] == 'Vertical Up':
        mu = row['Bend COF']
        theta_degrees = row['Bend Angle (°)']
        R = row['Bend Radius (m)']
        if row['Bend Direction'] == 'Up':
            T_in = tension_out_vertical_bend_up_pulling_up(T_in, w, mu, theta_degrees, R, W)
        else:
            T_in = tension_out_vertical_bend_up_pulling_down(T_in, w, mu, theta_degrees, R, W)

    elif row['Bend Type'] == 'Vertical Down':
        mu = row['Bend COF']
        theta_degrees = row['Bend Angle (°)']
        R = row['Bend Radius (m)']
        if row['Bend Direction'] == 'Down':
            T_in = tension_out_vertical_bend_down_pulling_down(T_in, w, mu, theta_degrees, R, W)
        else:
            T_in = tension_out_vertical_bend_down_pulling_up(T_in, w, mu, theta_degrees, R, W)

    if is_slope_zero_or_nan and is_direction_nan and not is_length_zero_or_nan:

        mu = row['Straight Section COF']
        return (w * mu * W * length)*9.81/1000 + T_in  ## Output kN

    elif not is_slope_zero_or_nan:
        mu = row['Straight Section COF']
        phi_degrees = slope
        if direction == 'Up':
            return incline_tension_out_up(T_in, W, length, phi_degrees, w, mu) ## Output kN
        else:
            return incline_tension_out_down(T_in, W, length, phi_degrees, w, mu) ## Output kN
    if i == 0 and is_slope_zero_or_nan and is_direction_nan and is_length_zero_or_nan:
        return round((Back_Tension), 3) ## Output kN
    elif i != 0 and is_slope_zero_or_nan and is_direction_nan and is_length_zero_or_nan:
        return T_in

  ###################### INcline Cases #################################

def update_tension_and_pressure_reverse_case(df, Back_Tension, w, W, Cable_Configuration):
    j = 0
    i = 0
    for j in range(len(df)):
        df.at[j, 'Tension'] = calculate_tension_reverse_case(df.iloc[j], Back_Tension, w, W, i, df)
        i += 1
    add_side_wall_pressure(df, Cable_Configuration, w)
    return df

##[Beta]

In [417]:
def handle_joint_bays(df):
    """
    Handles the 'Joint Bay' column in the DataFrame.
    - If the 'Joint Bay' column does not exist, it adds it.
    - Informs the user how many joint bays are already located in the DataFrame.
    - Asks if the user would like to add more joint bays.
    - Repositions the 'Joint Bay' column immediately after the 'wh' column.

    Args:
        df (pandas.DataFrame): The DataFrame to process.

    Returns:
        pandas.DataFrame: The updated DataFrame with 'Joint Bay' column populated and positioned.
        pandas.DataFrame: A DataFrame containing the Joint Bay locations and corresponding 'CH' values.
    """

    # Check if df is a valid DataFrame
    if df is None or not isinstance(df, pd.DataFrame):
        raise ValueError("Input 'df' must be a valid pandas DataFrame.")

    # Ensure 'Joint Bay' column exists
    if 'Joint Bay' not in df.columns:
        print("'Joint Bay' column not found. Adding 'Joint Bay' column.")
        df['Joint Bay'] = np.nan  # Add new 'Joint Bay' column with NaN
    else:
        # Get rows where 'Joint Bay' is already populated
        existing_joint_bays = df[df['Joint Bay'].notna()].index.tolist()
        num_existing_bays = len(existing_joint_bays)

        if num_existing_bays > 0:
            # Inform the user about the existing joint bay locations
            print(f"There are {num_existing_bays} joint bays already populated in rows: {existing_joint_bays}")
            # Ask the user if they want to add more joint bays
            add_more = input("Would you like to add more joint bays? (yes/no): ").strip().lower()
            if add_more != 'yes':
                print("No additional joint bays will be added.")
                return df, df[df['Joint Bay'].notna()][['Joint Bay', 'CH']] if 'CH' in df.columns else df[df['Joint Bay'].notna()][['Joint Bay']]

    # Prompt user to input joint bay locations and names only for rows where 'Joint Bay' is NaN
    populate_joint_bay_column(df)

    # After handling, retrieve joint bay locations
    if 'CH' in df.columns:
        joint_bay_locations = df[df['Joint Bay'].notna()][['Joint Bay', 'CH']]
    else:
        joint_bay_locations = df[df['Joint Bay'].notna()][['Joint Bay']]

    # Reposition the 'Joint Bay' column immediately after the 'wh' column
    if 'wh' in df.columns:
        cols = df.columns.tolist()
        # Remove 'Joint Bay' column if it exists, and re-insert it after 'wh'
        joint_bay_col = cols.pop(cols.index('Joint Bay'))  # Remove Joint Bay column
        wh_index = cols.index('wh')  # Find where 'wh' is located
        cols.insert(wh_index + 1, joint_bay_col)  # Insert Joint Bay after 'wh'
        df = df[cols]  # Reorder the DataFrame with the new column order
    else:
        print("'wh' column not found. 'Joint Bay' column position not changed.")

    return df, joint_bay_locations

def populate_joint_bay_column(df):
    """
    Helper function to populate the 'Joint Bay' column based on user input.
    Only prompts for input where 'Joint Bay' is missing (NaN).
    Populates the column in bulk rather than row by row.

    Args:
        df (pandas.DataFrame): The DataFrame to update.
    """

    # Ask the user how many joint bays they want to include (only if NaN entries exist)
    num_nan_rows = df['Joint Bay'].isna().sum()
    if num_nan_rows == 0:
        print("All 'Joint Bay' entries are already populated.")
        return

    print(f"There are {num_nan_rows} rows where 'Joint Bay' is missing.")
    while True:
        try:
            num_joint_bays = int(input(f"Enter the number of joint bays to populate (max {num_nan_rows}): "))
            if num_joint_bays <= 0 or num_joint_bays > num_nan_rows:
                print(f"Please enter a positive integer between 1 and {num_nan_rows}.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter an integer.")

    # Display available row numbers with NaN values
    joint_bay_rows = df[df['Joint Bay'].isna()].index.tolist()  # Get indices of NaN rows
    print(f"Available rows for new joint bays: {joint_bay_rows}")

    # Get all row numbers for the joint bays
    while True:
        try:
            input_rows = input(f"Enter {num_joint_bays} row numbers (separated by commas) from the list above: ")
            joint_bay_rows_input = list(map(int, input_rows.split(',')))

            # Validate that the entered rows are valid and within the allowed rows
            if len(joint_bay_rows_input) != num_joint_bays or not all(row in joint_bay_rows for row in joint_bay_rows_input):
                print(f"Please enter {num_joint_bays} valid row numbers from the available rows.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter valid integers.")

    # Ask the user for joint bay names in bulk
    joint_bay_names = []
    for i in range(num_joint_bays):
        joint_bay_name = input(f"Enter the name for joint bay at row {joint_bay_rows_input[i]}: ")
        joint_bay_names.append(joint_bay_name)

    # Populate the 'Joint Bay' column with the provided names
    for row, name in zip(joint_bay_rows_input, joint_bay_names):
        df.at[row, 'Joint Bay'] = name

    # Retrieve the 'CH' values corresponding to the joint bay rows
    if 'CH' in df.columns:
        joint_bay_locations = df.loc[joint_bay_rows_input, ['Joint Bay', 'CH']]
        print("\nJoint Bay Locations and Corresponding 'CH' Values:")
        print(joint_bay_locations)
    else:
        print("'CH' column not found in the DataFrame.")
        joint_bay_locations = df.loc[joint_bay_rows_input, ['Joint Bay']]



#Input Data

##SPECIFIC WEIGHT

In [ ]:
while True:
    # Prompt the user to enter the weight, reminding them to include the unit
    input_value = input("Enter the total cable assembly weight per unit of length (Kg/m or kN/m). Don't forget to include the units: ")

    # Check if the user entered both a value and a unit
    if len(input_value.split()) == 2:
        try:
            # Split the input into value and unit3
            value, unit = input_value.split()

            # Convert the value to float
            value = float(value)

            # Initialize a variable to store the weight in kg/m
            W = 0.0

            # Check the unit and convert if necessary
            if unit.lower() == "kg/m":
                W = value  # Already in kg/m
            elif unit.lower() == "kn/m":
                W = value / (9.80665 / 1000)  # Convert kN/m to kg/m
            else:
                print("Invalid unit entered. Please use either 'Kg/m' or 'kN/m'.")
                continue  # Go back to the start of the loop

            # Round the weight to two decimal places
            W = round(W, 2)

            # If reached here, the input is valid
            print(f"Stored weight in kg/m: {W}")
            break  # Exit the loop

        except ValueError:
            print("Invalid numeric value entered. Please try again.")
    else:
        print("Please enter both the value and the unit, separated by a space.")


Enter the total cable assembly weight per unit of length (Kg/m or kN/m). Don't forget to include the units: 4.06 kg/m
Stored weight in kg/m: 4.06


##MAX TENSION

In [ ]:
Max_Tension = get_max_tension()

Please enter the Maximum Allowable Tension in kN: 20
Stored Maximum Allowable Tension: 20.0 kN


##MAX Side Wall Pressure [SWP]

In [ ]:
max_SWP = get_max_side_wall_pressure()

Enter the Max Side Wall Pressure (in kN/m): 10

The Max Side Wall Pressure is 10.00 kN/m


##BACK TENSION

In [ ]:
while True:
    # Prompt the user to enter back tension in kN, mentioning that it is usually 0.5% to 3% of the total weight
    input_value = input("Enter the back tension in kN (usually a value between 0.5% to 3% of the total weight): ")

    try:
        # Convert the input to float
        Back_Tension = float(input_value)


        # If reached here, the input is valid
        print(f"Stored back tension in kN: {Back_Tension}")
        break  # Exit the loop

    except ValueError:
        print("Invalid numeric value entered. Please try again.")



Enter the back tension in kN (usually a value between 0.5% to 3% of the total weight): 0.5
Stored back tension in kN: 0.5


##WEIGHT CORRELATION FACTOR

In [ ]:
User_inputs=calculate_wcf()

FloatText(value=0.0, description='Duct Diameter [D] (mm):', style=DescriptionStyle(description_width='initial'…

FloatText(value=0.0, description='Cable Diameter [d] (mm):', style=DescriptionStyle(description_width='initial…

Dropdown(description='Configuration:', options=(('1 Cable', 1), ('2 Cables', 2), ('3 Cables (Triangular)', 3),…

Button(description='Calculate WCF', style=ButtonStyle())

Output()

###Assign variables to user input

In [ ]:
w =User_inputs[0]
Cable_Configuration = User_inputs[1]
Duct_Diameter = User_inputs[2]
Cable_Diameter = User_inputs[3]

###Overwrite WEIGHT CORRELATION FACTOR

In [ ]:
while True:
    # Prompt the user to enter the weight correlation factor
    input_value = input("Enter the weight correlation factor (dimensionless): ")

    try:
        # Try converting the input to a float
        w = float(input_value)

        # If the input is successfully converted, it's valid
        print(f"Stored weight correlation factor: {w}")
        break  # Exit the loop
    except ValueError:
        # If the conversion to float fails, prompt the user to try again
        print("Invalid input. Please enter a number.")


#COF

##COF Data Base[Unlubricated]

##Load data baese.

In [ ]:
COF_Data_Base_path = '/content/drive/MyDrive/Colab Notebooks/COF Data Base/COF Data Base.csv'

COF_Data_Base = try_reading_file(COF_Data_Base_path)

# If that doesn't work, try manual decode (usually optional)
if COF_Data_Base is None:
    ts = manually_decode_and_read(COF_Data_Base)

Successfully read the file with utf-8 encoding.


In [ ]:
get_cof_and_source(COF_Data_Base)

Dropdown(description='Duct Material:', options=('PVC', 'Rigid Steel', 'EMT', 'HDPE', 'Fiberglass', 'Aluminium'…

Dropdown(description='Cable Material:', options=('CPE', 'HDPE', 'LLDPE', 'Nylon (THHN)', 'PVC', 'XLPE', 'TPR',…

Button(description='Get COF and Source', style=ButtonStyle())

COF: 0.39, Duct Material: PVC, Cable Material: HDPE, Source: Polywater -Pullplanner


##Assign COF to Data Frame

In [418]:
# Main logic
while True:
    choice = input("Would you like to use a constant coefficient of friction (c) or a variable one for different sections (v)? Enter 'c' or 'v': ").lower()
    if choice in ['c', 'v']:
        break
    else:
        print("Invalid choice. Please enter 'c' or 'v'.")

if choice == 'c':
    mu_value = float(input("Please enter the constant coefficient of friction value: "))

    # Check if "Straight Section COF" column exists before adding it
    if "Straight Section COF" not in df.columns:
        # Add a new column "Straight Section COF" next to "Straight Section Length (m)"
        df.insert(df.columns.get_loc("Straight Section Length (m)") + 1, "Straight Section COF", None)

    # Check if "Bend COF" column exists before adding it
    if "Bend COF" not in df.columns:
        # Add a new column "Bend COF" next to "Arc Length"
        df.insert(df.columns.get_loc("Arc Length") + 1, "Bend COF", None)

    # Replace all rows in "Straight Section COF" and "Bend COF" with the input value
    df["Straight Section COF"] = mu_value
    df["Bend COF"] = mu_value

else:
    # Check if "Straight Section COF" column exists before adding it
    if "Straight Section COF" not in df.columns:
        # Add a new column "Straight Section COF" next to "Straight Section Length (m)"
        df.insert(df.columns.get_loc("Straight Section Length (m)") + 1, "Straight Section COF", None)

    # Check if "Bend COF" column exists before adding it
    if "Bend COF" not in df.columns:
        # Add a new column "Bend COF" next to "Arc Length"
        df.insert(df.columns.get_loc("Arc Length") + 1, "Bend COF", None)

Would you like to use a constant coefficient of friction (c) or a variable one for different sections (v)? Enter 'c' or 'v': c
Please enter the constant coefficient of friction value: 0.20


### Edit COF to a specific value


In [ ]:
sheet = sheets.InteractiveSheet(df=df)

#### Check Edited data frame

In [ ]:
df = sheet.as_df()
df

#Calcs

In [419]:
df, joint_bay_locations = handle_joint_bays(df)
df

'Joint Bay' column not found. Adding 'Joint Bay' column.
There are 40 rows where 'Joint Bay' is missing.
Enter the number of joint bays to populate (max 40): 2
Available rows for new joint bays: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
Enter 2 row numbers (separated by commas) from the list above: 0, 39
Enter the name for joint bay at row 0: JB1
Enter the name for joint bay at row 39: JB2
'CH' column not found in the DataFrame.


<ipython-input-417-5e9190b0c81d>:114: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'JB1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



,wh,Joint Bay,Straight Section Slope (°),Slope Direction,Straight Section Length (m),Straight Section COF,Bend Type,Bend Direction,Bend Radius (m),Bend Angle (°),Arc Length,Bend COF
0,1,JB1,0.0,NaN,8.3,0.2,NaN,NaN,0.00,0.0,0.0,0.2
1,2,NaN,0.0,NaN,6.6,0.2,Vertical Up,Up,10.00,1.3,0.2,0.2
2,3,NaN,0.0,NaN,0.0,0.2,Horizontal,NaN,88.94,11.5,17.9,0.2
3,4,NaN,0.0,NaN,1.7,0.2,NaN,NaN,0.00,0.0,0.0,0.2
4,5,NaN,0.0,NaN,9.9,0.2,Horizontal,NaN,62.37,12.6,13.7,0.2
5,6,NaN,0.0,NaN,16.2,0.2,NaN,NaN,0.00,0.0,0.0,0.2
6,7,NaN,0.0,NaN,72.9,0.2,NaN,NaN,0.00,0.0,0.0,0.2
7,8,NaN,0.0,NaN,105.4,0.2,Vertical Down,Down,5.00,9.8,0.9,0.2
8,9,NaN,9.0,Down,5.2,0.2,Vertical Up,Down,5.00,9.3,0.8,0.2
9,10,NaN,0.0,NaN,0.0,0.2,Vertical Down,Down,5.00,9.4,0.8,0.2


##Define Calcs Boundaries

In [420]:
Results_1 = create_and_clean_subset(df)

Available Joint Bays: ['JB1', 'JB2']
Enter the starting Joint Bay: JB1
Enter the ending Joint Bay: JB2


##Results

In [422]:
Results_1  = combined_function(Results_1 , Back_Tension, w, W, Cable_Configuration)
Results_1
calculate_length_add_previous(Results_1)
Results_1 = round_to_two_decimals(Results_1)
Results_1

<ipython-input-416-0465e93eef07>:600: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,wh,Joint Bay,Straight Section Slope (°),Slope Direction,Straight Section Length (m),Straight Section COF,Bend Type,Bend Direction,Bend Radius (m),Bend Angle (°),Arc Length,Bend COF,Tension,Side Wall Pressure,Reverse Tension,Reverse Side Wall Pressure,Length (m)
0,1,JB1,0.0,NaN,8.3,0.2,NaN,NaN,0.00,0.0,0.0,0.2,0.80,NaN,8.04,NaN,8.30
1,2,NaN,0.0,NaN,6.6,0.2,Vertical Up,Up,10.00,1.3,0.2,0.2,0.85,0.09,7.97,0.80,15.13
2,3,NaN,0.0,NaN,0.0,0.2,Horizontal,NaN,88.94,11.5,17.9,0.2,1.00,0.01,7.88,0.09,32.98
3,4,NaN,0.0,NaN,1.7,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.01,NaN,7.54,NaN,34.68
4,5,NaN,0.0,NaN,9.9,0.2,Horizontal,NaN,62.37,12.6,13.7,0.2,1.21,0.02,7.53,0.12,58.29
5,6,NaN,0.0,NaN,16.2,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.34,NaN,7.11,NaN,74.49
6,7,NaN,0.0,NaN,72.9,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.92,NaN,6.98,NaN,147.39
7,8,NaN,0.0,NaN,105.4,0.2,Vertical Down,Down,5.00,9.8,0.9,0.2,2.86,0.57,6.40,1.28,253.65
8,9,NaN,9.0,Down,5.2,0.2,Vertical Up,Down,5.00,9.3,0.8,0.2,2.95,0.59,5.36,1.07,259.66
9,10,NaN,0.0,NaN,0.0,0.2,Vertical Down,Down,5.00,9.4,0.8,0.2,3.06,0.61,5.11,1.02,260.48


#Suggested Back Tension

In [423]:
Back_Tension = suggested_back_tension(Results_1, W)

Enter the percentage of total weight for back tension (e.g., 3 for 3%): 3
Suggested Back Tension: 0.73 kN based on the total length value of 613.34 m


##Recalculate with Suggested Back Tension

In [426]:
Results_1  = combined_function(Results_1, Back_Tension, w, W, Cable_Configuration)
Results_1 = round_to_two_decimals(Results_1)

<ipython-input-416-0465e93eef07>:600: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



##Cable Pusher

In [427]:
Back_Tension_and_Cable_Pusher = adjust_back_tension_for_cable_pusher(Back_Tension)
Back_Tension_and_Cable_Pusher

Enter the value for the cable pusher in kN: 0


(0.73, 0.0)

In [428]:
Results_1  = combined_function(Results_1, Back_Tension_and_Cable_Pusher[0], w, W, Cable_Configuration)
Results_1

,wh,Joint Bay,Straight Section Slope (°),Slope Direction,Straight Section Length (m),Straight Section COF,Bend Type,Bend Direction,Bend Radius (m),Bend Angle (°),Arc Length,Bend COF,Tension,Side Wall Pressure,Reverse Tension,Reverse Side Wall Pressure,Length (m)
0,1,JB1,0.0,NaN,8.3,0.2,NaN,NaN,0.00,0.0,0.0,0.2,0.796,NaN,8.035115,NaN,8.30
1,2,NaN,0.0,NaN,6.6,0.2,Vertical Up,Up,10.00,1.3,0.2,0.2,0.854,0.085400,7.968574,0.796857,15.13
2,3,NaN,0.0,NaN,0.0,0.2,Horizontal,NaN,88.94,11.5,17.9,0.2,1.001,0.011255,7.884000,0.088644,32.98
3,4,NaN,0.0,NaN,1.7,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.015,NaN,7.542542,NaN,34.68
4,5,NaN,0.0,NaN,9.9,0.2,Horizontal,NaN,62.37,12.6,13.7,0.2,1.214,0.019464,7.528861,0.120713,58.29
5,6,NaN,0.0,NaN,16.2,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.343,NaN,7.112045,NaN,74.49
6,7,NaN,0.0,NaN,72.9,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.924,NaN,6.982701,NaN,147.39
7,8,NaN,0.0,NaN,105.4,0.2,Vertical Down,Down,5.00,9.8,0.9,0.2,2.864,0.572800,6.401587,1.280317,253.65
8,9,NaN,9.0,Down,5.2,0.2,Vertical Up,Down,5.00,9.3,0.8,0.2,2.954,0.590800,5.362000,1.072400,259.66
9,10,NaN,0.0,NaN,0.0,0.2,Vertical Down,Down,5.00,9.4,0.8,0.2,3.056,0.611200,5.112000,1.022400,260.48


#Lubricant

In [429]:
Lubricant = Results_1 ['Length (m)'].iloc[-1] * 0.0080 * Cable_Diameter/10
Lubricant = round(Lubricant, 2)
print(f"Suggested Lubricant Amount: {Lubricant} lts")

Suggested Lubricant Amount: 30.91 lts


#Results with notes

In [430]:
Results_1 = add_notes_column(Results_1, Back_Tension_and_Cable_Pusher[1], Lubricant, Max_Tension, max_SWP, w, Cable_Configuration, additional_notes=None)
Results_1 = round_to_two_decimals(Results_1)
Results_1

<ipython-input-416-0465e93eef07>:600: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,wh,Joint Bay,Straight Section Slope (°),Slope Direction,Straight Section Length (m),Straight Section COF,Bend Type,Bend Direction,Bend Radius (m),Bend Angle (°),Arc Length,Bend COF,Tension,Side Wall Pressure,Reverse Tension,Reverse Side Wall Pressure,Length (m),Notes
0,1,JB1,0.0,NaN,8.3,0.2,NaN,NaN,0.00,0.0,0.0,0.2,0.80,NaN,8.04,NaN,8.30,Cable Pusher: 0.0 kN
1,2,NaN,0.0,NaN,6.6,0.2,Vertical Up,Up,10.00,1.3,0.2,0.2,0.85,0.09,7.97,0.80,15.13,Lubricant: 30.91 lts
2,3,NaN,0.0,NaN,0.0,0.2,Horizontal,NaN,88.94,11.5,17.9,0.2,1.00,0.01,7.88,0.09,32.98,Max Side Wall Pressure: 1.83 kN/m
3,4,NaN,0.0,NaN,1.7,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.01,NaN,7.54,NaN,34.68,Max SWP Utilization Factor: 18.3 %
4,5,NaN,0.0,NaN,9.9,0.2,Horizontal,NaN,62.37,12.6,13.7,0.2,1.21,0.02,7.53,0.12,58.29,Max Forward Tension: 10.685 kN
5,6,NaN,0.0,NaN,16.2,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.34,NaN,7.11,NaN,74.49,Forward Utilization Factor: 53.42 %
6,7,NaN,0.0,NaN,72.9,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.92,NaN,6.98,NaN,147.39,Max Reverse Tension: 8.035115476 kN
7,8,NaN,0.0,NaN,105.4,0.2,Vertical Down,Down,5.00,9.8,0.9,0.2,2.86,0.57,6.40,1.28,253.65,Reverse Utilization Factor: 40.18 %
8,9,NaN,9.0,Down,5.2,0.2,Vertical Up,Down,5.00,9.3,0.8,0.2,2.95,0.59,5.36,1.07,259.66,Weight Correlation Factor: 1
9,10,NaN,0.0,NaN,0.0,0.2,Vertical Down,Down,5.00,9.4,0.8,0.2,3.06,0.61,5.11,1.02,260.48,Cable Configuration: 1 Cable


#COF Based on Target Tension

In [ ]:
Target_COf = adjust_cof_to_target_tension(Results_1, combined_function, Back_Tension, w, W, Cable_Configuration)
add_notes_column(Results_1, Back_Tension_and_Cable_Pusher[1], Lubricant, Max_Tension, max_SWP, w, Cable_Configuration, additional_notes=None)

#Graphs

In [432]:
# Assuming your DataFrame is named Results_1
# Calculate the variation in 'Tension' with respect to the previous value
Results_1['Tension Variation'] = Results_1['Tension'].diff()

# Check if the 'CH' column exists, otherwise use 'wh'
hover_column = 'CH' if 'CH' in Results_1.columns else 'wh'

# Create a 2D scatter plot with Side Wall Pressure and Tension Variation
fig = px.scatter(
    Results_1,
    x='Side Wall Pressure',
    y='Tension Variation',
    color='Tension Variation',
    color_continuous_scale='Viridis',
    hover_data={hover_column: True},  # Use the selected column in hover_data
    title='2D Scatter Plot of Side Wall Pressure and Tension Variation'
)

# Show the figure
fig.show()


In [433]:
# Reverse the order of the DataFrame to calculate diff from bottom to top
Results_1_reversed = Results_1.iloc[::-1].reset_index(drop=True)

# Calculate the variation in 'Reverse Tension' with respect to the previous value
Results_1_reversed['Reverse Tension Variation'] = Results_1_reversed['Reverse Tension'].diff()

# Restore the original order
Results_1_reversed = Results_1_reversed.iloc[::-1].reset_index(drop=True)

# Check if the 'CH' column exists, otherwise use 'wh'
hover_column = 'CH' if 'CH' in Results_1_reversed.columns else 'wh'

# Create a 2D scatter plot with Reverse Side Wall Pressure and Reverse Tension Variation
fig = px.scatter(
    Results_1_reversed,
    x='Reverse Side Wall Pressure',
    y='Reverse Tension Variation',
    color='Reverse Tension Variation',
    color_continuous_scale='Viridis',
    hover_data={hover_column: True},  # Use the selected column in hover_data
    title='2D Scatter Plot of Reverse Side Wall Pressure and Reverse Tension Variation'
)

# Customize layout to match the style of the provided image
fig.update_layout(
    plot_bgcolor='rgba(240, 240, 240, 1)',  # Adjust the background color if necessary
    xaxis_title='Reverse Side Wall Pressure',
    yaxis_title='Reverse Tension Variation',
    coloraxis_colorbar=dict(
        title="Tension Variation"
    )
)

# Show the figure
fig.show()

# Restore Results_1 to the reversed version (if needed for later calculations)
Results_1 = Results_1_reversed


In [434]:
Results_1

,wh,Joint Bay,Straight Section Slope (°),Slope Direction,Straight Section Length (m),Straight Section COF,Bend Type,Bend Direction,Bend Radius (m),Bend Angle (°),Arc Length,Bend COF,Tension,Side Wall Pressure,Reverse Tension,Reverse Side Wall Pressure,Length (m),Notes,Tension Variation,Reverse Tension Variation
0,1,JB1,0.0,NaN,8.3,0.2,NaN,NaN,0.00,0.0,0.0,0.2,0.80,NaN,8.04,NaN,8.30,Cable Pusher: 0.0 kN,NaN,0.07
1,2,NaN,0.0,NaN,6.6,0.2,Vertical Up,Up,10.00,1.3,0.2,0.2,0.85,0.09,7.97,0.80,15.13,Lubricant: 30.91 lts,0.05,0.09
2,3,NaN,0.0,NaN,0.0,0.2,Horizontal,NaN,88.94,11.5,17.9,0.2,1.00,0.01,7.88,0.09,32.98,Max Side Wall Pressure: 1.83 kN/m,0.15,0.34
3,4,NaN,0.0,NaN,1.7,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.01,NaN,7.54,NaN,34.68,Max SWP Utilization Factor: 18.3 %,0.01,0.01
4,5,NaN,0.0,NaN,9.9,0.2,Horizontal,NaN,62.37,12.6,13.7,0.2,1.21,0.02,7.53,0.12,58.29,Max Forward Tension: 10.685 kN,0.20,0.42
5,6,NaN,0.0,NaN,16.2,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.34,NaN,7.11,NaN,74.49,Forward Utilization Factor: 53.42 %,0.13,0.13
6,7,NaN,0.0,NaN,72.9,0.2,NaN,NaN,0.00,0.0,0.0,0.2,1.92,NaN,6.98,NaN,147.39,Max Reverse Tension: 8.035115476 kN,0.58,0.58
7,8,NaN,0.0,NaN,105.4,0.2,Vertical Down,Down,5.00,9.8,0.9,0.2,2.86,0.57,6.40,1.28,253.65,Reverse Utilization Factor: 40.18 %,0.94,1.04
8,9,NaN,9.0,Down,5.2,0.2,Vertical Up,Down,5.00,9.3,0.8,0.2,2.95,0.59,5.36,1.07,259.66,Weight Correlation Factor: 1,0.09,0.25
9,10,NaN,0.0,NaN,0.0,0.2,Vertical Down,Down,5.00,9.4,0.8,0.2,3.06,0.61,5.11,1.02,260.48,Cable Configuration: 1 Cable,0.11,0.18


# Download Results

In [ ]:
download_dataframe(Results_1)

Do you want to download the file? Type 'y' for yes or 'n' for no: y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [435]:
#To Do
#Add check length in the data format excel
#add keep record of the files
#Tranning data, test pull vs actual + data points form CAD